![](https://d33wubrfki0l68.cloudfront.net/d04566d0f6671ae94fdae6fa3f767f5a6553d335/c50f0/blog/img/spacy-pytorch-transformers.jpg)

## Experiment: How does a model trained on Prodigy annotations compare to a plain spaCy model trained on the same seed terms?  

**training data**
- This task is simlar to creating the JSONL seed words, but in this case, we're going to create training data with a section of text and the indexes for the new entity in that text.
- This is very simlar to the standoff text, so it's just a matter of parsing the text at the sentence level and noting where the new label appears in the text. 
```json
[
    (
        "Horses are too tall and they pretend to care about your feelings",
        {"entities": [(0, 6, LABEL)]},
    ), ...
]
```

In [1]:
import os
import en_core_web_sm
import json
import random 
import pickle
import spacy
import standoffconverter
from lxml import etree
from spacy.tokens import Doc
from urllib.request import urlopen

nlp = spacy.load('en_core_web_sm')
ner = nlp.get_pipe("ner")
ner.add_label("PLACE")
#nlp = spacy.blank("en")
#nlp.create_pipe('ner')
#nlp.add_pipe(ner)
#ner = nlp.get_pipe("ner")

def get_tei_standoff(ref):
    
    url = 'http://www.perseus.tufts.edu/hopper/xmlchunk?doc=' + ref
    print(url)

    tei = urlopen(url).read()
    tei = etree.XML(tei)
    markup = standoffconverter.tree_to_standoff(tei)

    return markup

refs = pickle.load(open('refs.pickle', 'rb'))

TRAIN_DATA = []
labels = []
patterns = []

for ref in refs:
    ents_dict = {}
    ents_dict['entities'] = []
    standoff = get_tei_standoff(ref)

    text = standoff[0]

    for tag in standoff[1]:
        try:
            if tag['attrib']['type'] == 'place':
                word_start = tag['begin']
                word_end = tag['end']
                type_ =  tag['attrib']['type']
                
                #Create patterns
                row = {}
                #{"label": "ORG", "pattern": "Apple"},
                word_start = tag['begin']
                word_end = tag['end']
                row['pattern'] = text[word_start:word_end]
                row['label'] = type_.upper()
                patterns.append(row)
                
                if type_ not in labels:
                    labels.append(type_)
                
                #Create training data
                #("Horses are too tall and they pretend to care about your feelings",{"entities": [(0, 6, LABEL)]},)
                ents_dict = {}
                ents_dict['entities'] = []
                entity = (word_start, word_end, type_.upper())
                #word_len = word_end - word_start
                #entity = (80, 80 + word_len, type_)
                ents_dict['entities'].append(entity)
                #ent_text = text[word_start - 80 : word_start + word_len + 80].replace('\n','')
                #TRAIN_DATA.append((ent_text, ents_dict))
        except Exception as e:
            pass
    TRAIN_DATA.append((text, ents_dict))

http://www.perseus.tufts.edu/hopper/xmlchunk?doc=Perseus%3Atext%3A1999.03.0070%3Anarrative%3D1
http://www.perseus.tufts.edu/hopper/xmlchunk?doc=Perseus%3Atext%3A1999.03.0070%3Anarrative%3D2
http://www.perseus.tufts.edu/hopper/xmlchunk?doc=Perseus%3Atext%3A1999.03.0070%3Anarrative%3D3
http://www.perseus.tufts.edu/hopper/xmlchunk?doc=Perseus%3Atext%3A1999.03.0070%3Anarrative%3D4
http://www.perseus.tufts.edu/hopper/xmlchunk?doc=Perseus%3Atext%3A1999.03.0070%3Anarrative%3D5
http://www.perseus.tufts.edu/hopper/xmlchunk?doc=Perseus%3Atext%3A1999.03.0070%3Anarrative%3D6
http://www.perseus.tufts.edu/hopper/xmlchunk?doc=Perseus%3Atext%3A1999.03.0070%3Anarrative%3D7
http://www.perseus.tufts.edu/hopper/xmlchunk?doc=Perseus%3Atext%3A1999.03.0070%3Anarrative%3D8
http://www.perseus.tufts.edu/hopper/xmlchunk?doc=Perseus%3Atext%3A1999.03.0070%3Anarrative%3D9
http://www.perseus.tufts.edu/hopper/xmlchunk?doc=Perseus%3Atext%3A1999.03.0070%3Anarrative%3D10
http://www.perseus.tufts.edu/hopper/xmlchunk?doc=

http://www.perseus.tufts.edu/hopper/xmlchunk?doc=Perseus%3Atext%3A1999.03.0070%3Anarrative%3D88
http://www.perseus.tufts.edu/hopper/xmlchunk?doc=Perseus%3Atext%3A1999.03.0070%3Anarrative%3D89
http://www.perseus.tufts.edu/hopper/xmlchunk?doc=Perseus%3Atext%3A1999.03.0070%3Anarrative%3D90
http://www.perseus.tufts.edu/hopper/xmlchunk?doc=Perseus%3Atext%3A1999.03.0070%3Anarrative%3D91
http://www.perseus.tufts.edu/hopper/xmlchunk?doc=Perseus%3Atext%3A1999.03.0070%3Anarrative%3D92
http://www.perseus.tufts.edu/hopper/xmlchunk?doc=Perseus%3Atext%3A1999.03.0070%3Anarrative%3D93
http://www.perseus.tufts.edu/hopper/xmlchunk?doc=Perseus%3Atext%3A1999.03.0070%3Anarrative%3D94
http://www.perseus.tufts.edu/hopper/xmlchunk?doc=Perseus%3Atext%3A1999.03.0070%3Anarrative%3D95
http://www.perseus.tufts.edu/hopper/xmlchunk?doc=Perseus%3Atext%3A1999.03.0070%3Anarrative%3D96
http://www.perseus.tufts.edu/hopper/xmlchunk?doc=Perseus%3Atext%3A1999.03.0070%3Anarrative%3D97
http://www.perseus.tufts.edu/hopper/xmlc

http://www.perseus.tufts.edu/hopper/xmlchunk?doc=Perseus%3Atext%3A1999.03.0070%3Anarrative%3D173
http://www.perseus.tufts.edu/hopper/xmlchunk?doc=Perseus%3Atext%3A1999.03.0070%3Anarrative%3D174
http://www.perseus.tufts.edu/hopper/xmlchunk?doc=Perseus%3Atext%3A1999.03.0070%3Anarrative%3D175
http://www.perseus.tufts.edu/hopper/xmlchunk?doc=Perseus%3Atext%3A1999.03.0070%3Anarrative%3D176
http://www.perseus.tufts.edu/hopper/xmlchunk?doc=Perseus%3Atext%3A1999.03.0070%3Anarrative%3D177
http://www.perseus.tufts.edu/hopper/xmlchunk?doc=Perseus%3Atext%3A1999.03.0070%3Anarrative%3D178
http://www.perseus.tufts.edu/hopper/xmlchunk?doc=Perseus%3Atext%3A1999.03.0070%3Anarrative%3D179
http://www.perseus.tufts.edu/hopper/xmlchunk?doc=Perseus%3Atext%3A1999.03.0070%3Anarrative%3D180
http://www.perseus.tufts.edu/hopper/xmlchunk?doc=Perseus%3Atext%3A1999.03.0070%3Anarrative%3D181
http://www.perseus.tufts.edu/hopper/xmlchunk?doc=Perseus%3Atext%3A1999.03.0070%3Anarrative%3D182
http://www.perseus.tufts.edu/h

http://www.perseus.tufts.edu/hopper/xmlchunk?doc=Perseus%3Atext%3A1999.03.0070%3Anarrative%3D259
http://www.perseus.tufts.edu/hopper/xmlchunk?doc=Perseus%3Atext%3A1999.03.0070%3Anarrative%3D260
http://www.perseus.tufts.edu/hopper/xmlchunk?doc=Perseus%3Atext%3A1999.03.0070%3Anarrative%3D261
http://www.perseus.tufts.edu/hopper/xmlchunk?doc=Perseus%3Atext%3A1999.03.0070%3Anarrative%3D262
http://www.perseus.tufts.edu/hopper/xmlchunk?doc=Perseus%3Atext%3A1999.03.0070%3Anarrative%3D263
http://www.perseus.tufts.edu/hopper/xmlchunk?doc=Perseus%3Atext%3A1999.03.0070%3Anarrative%3D264
http://www.perseus.tufts.edu/hopper/xmlchunk?doc=Perseus%3Atext%3A1999.03.0070%3Anarrative%3D265
http://www.perseus.tufts.edu/hopper/xmlchunk?doc=Perseus%3Atext%3A1999.03.0070%3Anarrative%3D266
http://www.perseus.tufts.edu/hopper/xmlchunk?doc=Perseus%3Atext%3A1999.03.0070%3Anarrative%3D267
http://www.perseus.tufts.edu/hopper/xmlchunk?doc=Perseus%3Atext%3A1999.03.0070%3Anarrative%3D268
http://www.perseus.tufts.edu/h

http://www.perseus.tufts.edu/hopper/xmlchunk?doc=Perseus%3Atext%3A1999.03.0070%3Anarrative%3D345
http://www.perseus.tufts.edu/hopper/xmlchunk?doc=Perseus%3Atext%3A1999.03.0070%3Anarrative%3D346
http://www.perseus.tufts.edu/hopper/xmlchunk?doc=Perseus%3Atext%3A1999.03.0070%3Anarrative%3D347
http://www.perseus.tufts.edu/hopper/xmlchunk?doc=Perseus%3Atext%3A1999.03.0070%3Anarrative%3D348
http://www.perseus.tufts.edu/hopper/xmlchunk?doc=Perseus%3Atext%3A1999.03.0070%3Anarrative%3D349
http://www.perseus.tufts.edu/hopper/xmlchunk?doc=Perseus%3Atext%3A1999.03.0070%3Anarrative%3D350
http://www.perseus.tufts.edu/hopper/xmlchunk?doc=Perseus%3Atext%3A1999.03.0070%3Anarrative%3D351
http://www.perseus.tufts.edu/hopper/xmlchunk?doc=Perseus%3Atext%3A1999.03.0070%3Anarrative%3D352
http://www.perseus.tufts.edu/hopper/xmlchunk?doc=Perseus%3Atext%3A1999.03.0070%3Anarrative%3D353
http://www.perseus.tufts.edu/hopper/xmlchunk?doc=Perseus%3Atext%3A1999.03.0070%3Anarrative%3D354
http://www.perseus.tufts.edu/h

http://www.perseus.tufts.edu/hopper/xmlchunk?doc=Perseus%3Atext%3A1999.03.0070%3Anarrative%3D431
http://www.perseus.tufts.edu/hopper/xmlchunk?doc=Perseus%3Atext%3A1999.03.0070%3Anarrative%3D432
http://www.perseus.tufts.edu/hopper/xmlchunk?doc=Perseus%3Atext%3A1999.03.0070%3Anarrative%3D433
http://www.perseus.tufts.edu/hopper/xmlchunk?doc=Perseus%3Atext%3A1999.03.0070%3Anarrative%3D434
http://www.perseus.tufts.edu/hopper/xmlchunk?doc=Perseus%3Atext%3A1999.03.0070%3Anarrative%3D435
http://www.perseus.tufts.edu/hopper/xmlchunk?doc=Perseus%3Atext%3A1999.03.0070%3Anarrative%3D436
http://www.perseus.tufts.edu/hopper/xmlchunk?doc=Perseus%3Atext%3A1999.03.0070%3Anarrative%3D437
http://www.perseus.tufts.edu/hopper/xmlchunk?doc=Perseus%3Atext%3A1999.03.0070%3Anarrative%3D438
http://www.perseus.tufts.edu/hopper/xmlchunk?doc=Perseus%3Atext%3A1999.03.0070%3Anarrative%3D439
http://www.perseus.tufts.edu/hopper/xmlchunk?doc=Perseus%3Atext%3A1999.03.0070%3Anarrative%3D440
http://www.perseus.tufts.edu/h

XMLSyntaxError: xmlParseEntityRef: no name, line 103, column 75 (<string>, line 103)

In [53]:
#https://github.com/explosion/spaCy/issues/3558#issuecomment-487953653

import re


def trim_entity_spans(data: list) -> list:
    """Removes leading and trailing white spaces from entity spans.

    Args:
        data (list): The data to be cleaned in spaCy JSON format.

    Returns:
        list: The cleaned data.
    """
    invalid_span_tokens = re.compile(r'\s')

    cleaned_data = []
    for text, annotations in data:
        entities = annotations['entities']
        valid_entities = []
        for start, end, label in entities:
            valid_start = start
            valid_end = end
            while valid_start < len(text) and invalid_span_tokens.match(
                    text[valid_start]):
                valid_start += 1
            try:
                while valid_end > 1 and invalid_span_tokens.match(
                        text[valid_end - 1]):
                    valid_end -= 1
            except IndexError:
                pass
            valid_entities.append([valid_start, valid_end, label])
        cleaned_data.append([text, {'entities': valid_entities}])

    return cleaned_data
TRAIN_DATA = trim_entity_spans(TRAIN_DATA)

In [77]:
patterns[1]

{'pattern': 'Realme of England', 'label': 'PLACE'}

In [78]:
TRAIN_DATA[5]

('\nOf the commodities of Portugal\n. The second Chapter.\n\n       THE Marchandy also of Portugal\n\n       By divers lands turne into sale.\n       Portugalers with us have trouth in hand:\n       Whose Marchandy commeth much into England.\n       They ben our friends, with their commodities,\n       And wee English passen into their countrees.\n       Her land hath wine, Osey, Ware, and Graine,\n       Figges, Reysins, Hony and Cordoweyne:\n       Dates, and Salt, Hides, and such Marchandy:\n       And if they would to Flanders passe for by,\n       They should not bee suffred ones ner twyes,\n       For supporting of our cruell enemies,\n       That is to say Flemings with her gyle:\n       For changeable they are in little while.\n       Then I conclude by reasons many moe,\n       If we suffred neither friend nor foe,\n       What so enemies, and so supporting\n       Passe for by us in time of werring,\n       Seth our friends will not ben in cause\n       Of our hindring, if re

In [ ]:
"""
* Source: https://spacy.io/usage/training#example-new-entity-type
* Training: https://spacy.io/usage/training
* NER: https://spacy.io/usage/linguistic-features#named-entities

Compatible with: spaCy v2.1.0+
Last tested with: v2.1.0
"""

import random
import json
from pathlib import Path
import spacy
from spacy.util import minibatch, compounding
from spacy.pipeline import EntityRuler


model = 'en_core_web_sm'
new_model_name = 'spacy_v_prodigy'
output_dir = '/home/ajanco/projects/spaCy_workshops/unit3/spacy_v_prodigy'

#with open('training.jsonl','r') as f:
#    TRAIN_DATA = json.loads(f.read())
    

def main(model=None, new_model_name="animal", output_dir=None, n_iter=1, labels=labels, patterns=patterns):
    """Set up the pipeline and entity recognizer, and train the new entity."""
    random.seed(0)
    if model is not None:
        nlp = spacy.load(model)  # load existing spaCy model
        print("Loaded model '%s'" % model)
    else:
        nlp = spacy.blank("en")  # create blank Language class
        print("Created blank 'en' model")
    # Add entity recognizer to model if it's not in the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if "ner" not in nlp.pipe_names:
        ner = nlp.create_pipe("ner")
        nlp.add_pipe(ner)
    # otherwise, get it, so we can add labels to it
    else:
        ner = nlp.get_pipe("ner")
    
        #ner.add_label(label)  # add new entity label to entity recognizer
    # Adding extraneous labels shouldn't mess anything up
    #ner.add_label("VEGETABLE")
    if model is None:
        optimizer = nlp.begin_training()
    else:
        optimizer = nlp.resume_training()
    move_names = list(ner.move_names)
    # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
    print('[*] adding label(s): ')
    for label in labels:
        print(label.upper())
        ner.add_label(label.upper())

    ruler = EntityRuler(nlp)
    ruler.add_patterns(patterns)
    nlp.add_pipe(ruler)


    with nlp.disable_pipes(*other_pipes):  # only train NER
        sizes = compounding(1.0, 4.0, 1.001)
        # batch up the examples using spaCy's minibatch
        for itn in range(n_iter):
            random.shuffle(TRAIN_DATA)
            batches = minibatch(TRAIN_DATA, size=sizes)
            losses = {}
            for batch in batches:
                texts, annotations = zip(*batch)
                nlp.update(texts, annotations, sgd=optimizer, drop=0.35, losses=losses)
            print("Losses", losses)

    # test the trained model
    test_text =  """The army marched from Konia to Kaiseria (Caesarea), and thence to Sivas, where the feast of the Korbân (sacrifice) was celebrated. Here Mustafâ Pâshâ, the emperor's favourite, was promoted to the rank of second vezir, and called into the divân. The army then continued its march to Erzerum. Besides tiie guns provided by the commander-in-chief, there were forty large guns dragged by two thousand pairs of buftaloes. The army entered the castle of Kazmaghan, and halted under the walls of Eriviin in the year 1044 (1634).  
"""
    doc = nlp(test_text)
    print("Entities in '%s'" % test_text)
    for ent in doc.ents:
        print(ent.label_, ent.text)

    # save model to output directory
    if output_dir is not None:
        output_dir = Path(output_dir)
        if not output_dir.exists():
            output_dir.mkdir()
        nlp.meta["name"] = new_model_name  # rename model
        nlp.to_disk(output_dir)
        print("Saved model to", output_dir)

            
main(model=model, new_model_name=new_model_name, output_dir=output_dir, labels=labels, patterns=patterns)


In [82]:
import spacy
import pickle
from spacy import displacy
places = pickle.load(open('places.pickle', 'rb'))

nlp = spacy.load("spacy_v_prodigy")
doc = nlp(
    """The army marched from Konia to Kaiseria (Caesarea), and thence to Sivas, where the feast of the Korbân (sacrifice) was celebrated. Here Mustafâ Pâshâ, the emperor's favourite, was promoted to the rank of second vezir, and called into the divân. The army then continued its march to Erzerum. Besides tiie guns provided by the commander-in-chief, there were forty large guns dragged by two thousand pairs of buftaloes. The army entered the castle of Kazmaghan, and halted under the walls of Eriviin in the year 1044 (1634).  
"""
)

counter = 0
for ent in doc.ents:
    if ent.text in places:
        print(ent.text, ent.start_char, ent.end_char, ent.label_)
        counter += 1

print(f"{counter} of the entities were in the training data")
displacy.render(doc, style="ent")

0 of the entities were in the training data


/home/ajanco/anaconda3/envs/spacy-gpu/lib/python3.7/runpy.py:193: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  "__main__", mod_spec)


()